# Motivation: monoids are useful in designing parallel algorithms

Consider a list of integers:

```
[2, 3, 9, 0, 0, 1]
```

Suppose we are interested in finding the sum of the elements in this list. A sequential algorithm would step through each element of the list, and keep track of the sum $S_i$ after $i$ elements have been processed. Initially, $S_0 = 0$, and when the next element is encountered, we add it to $S_0$ to get $S_1$, and finally $S_n$ will be returned, where $n$ is the length of the input list. In total, this algorithm would take $S_{n + 1}$ steps to execute.

A parallel algorithm does not seem hard to imagine. We can split the input into pieces, each of length $k$, and assig each piece to a separate thread. Each thread can sequentially sum its piece, giving us a list of numbers $n/k$ long. We can this list into pieces of length $k$, and then repeat the process until we have only one element. For example, the above sum list could be parallelized by splitting it into pieces of length $2$:

```

 [ 2, 3, 3, 0, 0, 1 ]
   │  │  │  │  │  │
   └─┬┘  └─┬┘  └─┬┘
     │     │     │
     5     3     1
     │     │     │
     └──┬──┘     │
        │        │
        8        1
        │        │
        └────┬───┘
             │
             9

```

The parallel solution would execute in $O(\textrm{log}_k(n))$ steps, which is an improvement over the $O(n)$ performance of the sequential algorithm. 

Now, suppose that we were interested in producing not only the total, but a *prefix sum*. Given a list of $n$ numbers, produce a list of $n$ numbers, so that the $i$th element of the list is sum of elements with index, $0, ..., i$ from the input list. The last element of such a list would be the the total. The sequential solution is easy: simply push each $S_i$ into a list, and return this, instead of only $S_{n}$. It requires a little more thought to parallelize the algorithm, however, because now there is some state dependency. If $L_n$ is the result list, then clearly $L_i$ depends upon $L_{i - 1}$, and it would seem then that the problem must be executed sequentially, as we must compute $L_{i - 1}$ to get $L_i$. However, a little more thought can convince us that we could still break the problem down into pieces, and combine the result of each piece into the final solution. Let `;` represent the "compose" operation, which combines the two prefix sum lists. The operation is very much like list or string concatenation, and the following cases are relevant:

```
0) (empty list composed with some other list) 
   [] ; [x_0, x_1, ..., x_n] 
== [x_0, x_1, ..., x_n]

   [x_0, x_1, ..., x_n] ; []
== [x_0, x_1, ..., x_n]
        
1) (two non-empty lists combined) 
   [x_0, x_1, ..., x_m] ; [y_0, y_1, ..., y_n] 
== [x_0, x_1, ..., x_m, y_0 + x_m, y_1 + x_m, ..., y_n + x_m]
        
2) ((non-)importance of order of combination)
   [x_0,  ..., x_l] ; ([y_0, ..., y_m] ; [z_0, ..., z_n])
== [x_0, ..., x_l] ; [y_0, ..., y_m, z_0 + y_m, ..., z_n + y_m]
== [x_0, ..., x_l, y_0 + x_l, ..., y_m + x_l, z_0 + y_m + x_l, z_1 + y_m + x_l, ..., z_n + y_m + x_l]
== [x_0, ..., x_l, y_0 + x_l, ..., y_m + x_l] ; [z_0, ..., z_n]
== ([x_0, ..., x_l] ; [y_0, ..., y_m]) ; [z_0, ..., z_n]
```

Note that the `;` operation is not **symmetric** (also known as **commutative**); that is `x ; y != y ; x`, unlike addition where `x + y = y + x`. With addition, `0 + x = x`, so $0$ is considered an **identity**, as operating with it always returns the other number back; similarly $\;$ has as "identity" an element `[]`. Finally, like addition, `;` is **associative**: the order of operations does not matter, so `x ; (y ; z) = (x ; y) ; z`. Now, with the `;` operation in hand, paralellizing prefix sum is just as easy, conceptually, as parallelizing total sum. 

In general, suppose we have a set $A$, and a binary operation on $A$. Then, the tuple $(*, A)$ is a **monoid** if and only if:

0. (`*` is closed on `A`): `*: (A \times A) \mapsto A` 
1. (associative) for any `x, y, z \in A`: `(x * y) * z = x * (y * z)`
2. (existence of identity) there exists an element `e in A` such that for all `x in A`: `e * x = x * e = x`

In this document, we'll explore to what extent it is possible to parallelize problems by repeating the trick we performed to parallelize the prefix sum: finding a suitable monoid that models the problem, which then means we can break down any given task into sub-tasks, which can be combined (due to associativity). 

# The Free Monoid

Suppose we have a set of "primitive" symbols, `P`, and then consider the set of strings `S` formed by stringing together elements in `P`. `P is_subset S`, because strings with only one elements elements are also strings. We can join two strings together ("concatenate"); let us symoblize it with the operator `;` We naturally grasp that string concatenation is associative, because for a given string: `a ; b ; c`, it doesn't matter whether we concatenated `a` and `b` first, or `b` and $c$ first, as long as we put together the elements in the order `a ; b ; c`. We also naturally grasp that `;` is closed on $S$: the result of joining together any two strings is also a string. We let the empty string (or the empty symbol) be considered an element of $P$. Clearly, joining the empty string to any other string does leaves the other string unchanged. Furthermore,  There is enough information to define the monoid `(\;, S)`:
0. (generator set) there is a subset `P` of `S`
1. (`\;` is closed on S): every concatenation of some finite number of primitive symbols from `P` is an element of `S` 
2. (`\;` is associative): because it doesn't matter which particular sub-strings are joined first, in order to prdouce the full string
3. (existence of identity): the empty string `e` is such that: `es = se = s` for any string `s in S`. 

We can strengthen condition 2, for associativity, further, by writing instead:
2. (`;` is associative over the primitive elements): for any `a, b, c in P`, `a; (b; c) = (a; b;) c`

From this version of condition 2, the more general condition of `;` being associative follows as a consequence of how non-primitive strings are generated. 

For convenience, we don't write the $\;$ operator, because it is easy to grasp that we are joining together strings. That is:
```
"A" ; "B" ; "C" == "ABC"
```
Similarly, the empty string can be ommitted when it is in the middle of a string (but it is still useful to denote it explicitly at other times:
```
ABeC == ABC
```

This particular monoid is called the **free monoid**, and it will play an important role in our explorations.

# Parentheses Strings

Consider the two parentheses symbols `(` and `)`, which we can join together to form longer strings for example, `)()((()(`. So, we have a free monoid, with primitive symbols `(, )` and the string concatenation operator `;`. 

We usually use `(` and `)` in pairs. A parentheses string is balanced, if each `(` has a matching `)`. Let us consider a monoid which has the same axioms as the free monoid, but with an additional rule that models this notion of parenthesis matching:
0. (generator set) `P = {(, )}`
1. (`;` is closed on S): by definition, because concatenating two parentheses strings gives another parentheses string
2. (`;` is associative): by definition, any three choices of single parentheses is associative
3. (existence of identity): by definition, the empty parentheses string `e` is such that for any parentheses string `s`: `es = se = s`
4. (`(` cancels `)`): `() = e`


With rule 4, we can define any parentheses string to be balanced if it can be reduced to the empty string. For example:
```
    (())
    { '(' cancels ')' }
==  (e)
==  ()
==  e
```

In general, a free monoid generated by a primitive set of size 2, `P = {p, q}` such that `p ; q = e` is called a **bicyclic monoid**. In the literature, it is often referred to as the "bicyclic semigroup"; a semigroup only has a closed, associative operator, but no identity, so every monoid is trivially a semigroup. 

# The Parentheses Matching Problem

We often use balanced parentheses strings to denote nested/hierarchical/tree-like structurse. The parentheses matching problem requires resolving this tree structure: each open bracket has an enclosing `(`, and each `)` has a matching `(`, and we want to find the indices of these enclosing/matching parentheses within the string. Here are some example parentheses strings, their tree structures (`e` marks an empty string, and it is also the default parent of the first `(` in the string), and the output produced by a program which resolves the tree structure (`-1` is the index of the root):

```
1) string:  <empty string>

   indices:     
   tree:    e 
   output: 

2) string:  ()
                (   )   
   indices:     0   1   
   tree:    e◄──(◄──)
                :   :   
   output:     -1   0   


2) string:  (())
                (   )   (   )
   indices:     0   1   2   3
   tree:    e◄──(◄──────────)
                ▲   :   :   :
                └───(◄──)   :
                :   :   :   :
   output:     -1   0   1   2


3) string:  ()()
                (   )   (   )
   indices:     0   1   2   3
   tree:    e◄──(◄──)   :   :
            ▲   :   :   :   :
            └───────────(◄──)
                :   :   :   :
   output:     -1   0   1   2

```

A sequential solution would step through each element in the input string, by using a stack to keep track of unmatched open parantheses as we step through the parentheses string. 

In [2]:
# A stack is like a list: we can push an element on top, and pop the topmost element. Python lists already have pop,
# so let us define the methods push (an alias for append), and last, for convenience.
class Stack(list):
    def push(self, x):
        self.append(x)
        
    def last(self):
        return self[-1]

In [3]:
from copy import copy

def stack_match(inp):
    state = Stack([-1])
    out = []
    states = []
    
    for ix, p in enumerate(inp):
        states.append(copy(state))
        out.append(state.last())
        if p == "(":
            state.push(ix)
        else:
            state.pop()
    
    return out, states

def print_out_states(out, states):
    out_str = f"output: {out}"
    index_str = ""
    k = 0
    for i, c in enumerate(out_str):
        if c == "," or c == "]":
            index_str += f"{k}"
            k += 1
        else:
            index_str += " "
    index_str = index_str[1:]
    print(index_str)
    print(out_str)
    r = "states: [\n"
    for i, state in enumerate(states):
        r += f"       {i}: {state},\n"
    r += "        ]"
    print(r)

In [4]:
out, states = stack_match("(()(()))")
print_out_states(out, states)

          0  1  2  3  4  5  6  7
output: [-1, 0, 1, 0, 3, 4, 3, 0]
states: [
       0: [-1],
       1: [-1, 0],
       2: [-1, 0, 1],
       3: [-1, 0],
       4: [-1, 0, 3],
       5: [-1, 0, 3, 4],
       6: [-1, 0, 3],
       7: [-1, 0],
        ]


# The (free) monoids hiding in the matching problem

Recall that our stack keeps track of unmatched `(` encountered as we traverse the parentheses string, and `(` are parent items in the corresponding tree structure of our parentheses string. So, there is a close connection between the state of the stack, and the output format we chose. The output allows us to reconstruct the state of the stack at any given step: starting from item $i$ in the output, if we follow the parents, we can reconstruct the state of the stack at the `i`th step. For example, follow the parents from `output[5]`, and keep track of the parents encountered so far:
```
out[5] == 4    [4]
out[4] == 3    [3, 4]
out[3] == 0    [0, 3, 4]
out[0] == -1   [-1, 0, 3, 4]
```
The stack at step `5` is precisely `[-1, 0, 3, 4]`. This observation suggests that a useful monoid for constructing a parallel algorithm might involve keeping track of the stack. We can then combine the stack from various steps to generate the output. 

Examining the stack at each step, notice that we can construct the $j$th stack, from the $i$th stack, for any $i < j$, by first performing a number of pops on the $i$th stack, and then a number of pushes. This suggests an element of our monoid needs to keep track of two pieces of information, pops and pushes: `Stk(num_pops, push_list)`. The identity must be `Stk(0, [])`: pop zero elements and push nothing. `Stk(0, [i])` models a `(` at position `i`, while `Stk(1, [])` models a `)`. 

Given two stack-transform elements `(a, xs)` and `(b, ys)`, how should they be combined? Suppose our stack is currently `S`. The first element will pop the stack `a` times, and then push `xs`. Now, the second element will pop the stack `b` times. Suppose `b >= len(xs)`, then we would pop all the indices in `xs` from `S`, and perform `(b - len(xs))` additional pops on `S`. In total, `S` would be popped `a + (b - len(xs))` times. Then, we would push on `ys`. So, the net transformation would be `Stk(a + (b - len(xs)), ys)`. If `b < len(xs)`, we effectively only push `xs[:len(xs) - b]` indices onto the stack (i.e. `xs` without the last `b` elements. Then, we push `ys`. So, the net transformation would be `Stk(a, xs[:(len(xs) - b)] + ys)`. Putting the two cases together:
```
Stk(a, xs) ; Stk(b, ys) == Stk(a + b - min(len(xs), b), xs[:max(0, len(xs) - b)] + ys)
``` 
For `Stk`s to form a monoid under `;` operation, we must verify that `;` is associative. The easiest way to do this is by noting that there is an 1-to-1 match between `(Stk, ;)` and the generic free monoid. `Stk` is generated by primitive elements: `Stk(0, [])` (the identity element), `Stk(1, [])`, and `Stk(0, [i])` for each `i in Naturals`. We can show that concatenation of any three primitive elements of `Stk` is associative, so `;` is associative, by induction, in general. 

There is a simpler monoid than `Stk` we can work with: `Stk` carries around its entire list of pushes, but we could omit the list as a whole, and only carry the number of pushes. Let `Bic(u, v)` (`Bic` because now we will have a bicyclic monoid), which denotes "pop `u` times" then "push `v` times". Note that: `Bic(Stk(a, xs)) == Bic(a, len(xs))`. The primitives are: `Bic(0, 0)` (identity), `Bic(0, 1)` (push 1), and `Bic(1, 0)` (pop 1). The concatenation operator is inspired by the concatenation operator for `Stk`:
```
Bic(a, x) ; Bic(b, y) == Bic(a + b - min(x, b), max(0, x - b) + y)
```

There is a slightly prettier way to write this:
```
Bic(a, x) ; Bic(b, y) == Bic(a + b - min(x, b), x + y - min(x, b))
```

To see that `max(0, x - b) + y == x + y - min(x, b)`, note first the following facts:

```
(a < b == -a > - b)
     a < b
     { definition of integers }
==  -a > -b

(definition of max)
    max(a, b)
==  if a > b then a else b fi

(definition of min)
    min(a, b)
==  if a < b then a else b fi

(function application distributes over if-then-else)
    if p then a else b fi
==  f(if p then a else b fi)
==  if p then f(a) else f(b) fi

(min(a, b) == -1 * max(-b, -a))
    min(a, b)
==  if a < b then a else b fi
    { function application distributes over if-then-else }
== -1 * (if a < b then -a else -b fi)
    {}
== -1 * (if -a > -b then -a else -b fi)
== -1 * max(-b, -a)

(min and max are shift-invariant)
    min(a, b) + c
==  if a < b then a else b fi + c
==  if a < b then a + c else b + c fi
    { < is shift-invariant }
==  if a + c < b + c then a + c else b + c fi
==  min(a + c, b + c)
```

Therefore:
```
    max(0, x - b) + y
    { max(-b, -a) == -1 * min(a, b) }
==  y - min(0, b - x)
    { min is shift-invariant, shift by -x }
==  y - (min(x, b) - x)
==  x + y - min(x, b)
```

Note that `Bic` has a 1-to-1 match with the bicyclic monoid. It has two non-identity primitive elements, and `Bic(0, 1) ; Bic(1, 0) == Bic(0, 0)`. It is easy to show that any combination of three primitive elements is associative, so the general associativity of `;` follows from this, by induction.

In [9]:
class Stk:
    def __init__(self, n_pops, pushes):
        self.n_pops = n_pops
        self.pushes = pushes
        
    def combine(self, rhs):
        my_pushes = len(self.pushes)
        new_pops = self.n_pops + rhs.n_pops - min(my_pushes, rhs.n_pops)
        new_pushes = self.pushes[:max(0, my_pushes - rhs.n_pops)] + rhs.pushes
        return StackMonoid(new_pops, new_pushes)
    
    def __str__(self):
        return f'({self.n_pops}, {self.pushes})'
    
    def __repr__(self):
        return self.__str__()
    
class Bic:
    def __init__(self, n_pops, n_pushes):
        self.n_pops = n_pops
        self.n_pushes = n_pushes
        
    def combine(self, rhs):
        # Bic(a, b) ; Bic(c, d) = Bic(a + c - min(b, c), b + d - min(b, c)) 
        min_bc = min(self.n_pushes, rhs.n_pops)
        new_pops = self.n_pops + rhs.n_pops - min_bc
        new_pushes = self.n_pushes + rhs.n_pushes - min_bc
        return Bic(new_pops, new_pushes)
    
    def __str__(self):
        return f'({self.n_pops}, {self.n_pushes})'
    
    def __repr__(self):
        return self.__str__()

# Reconstructing the output from concatenations of `Stk` or `Bic`

Let us consider an example: `(())`. Applying the sequential stack algorithm:

In [6]:
out, states = stack_match("(())")
print_out_states(out, states)

          0  1  2  3
output: [-1, 0, 1, 0]
states: [
       0: [-1],
       1: [-1, 0],
       2: [-1, 0, 1],
       3: [-1, 0],
        ]


Our parentheses string will map to a list of stack-transform elements: `[(0, [0]), (0, [1]), (1, []), (1[])]`. Our monoid is associative

In [7]:

def map_input(inp):
    r = []
    for ix, p in enumerate(inp):
        if p == '(':
            r.append(StackMonoid(0, [ix]))
        else:
            r.append(StackMonoid(1, []))
    return r
    
inp = "()()"
expected_out = [-1, 0, 1, 0]
inp = map_input(inp)

current = [StackMonoid(0, [])] + inp
states = [copy(current)]
while len(current) > 1:
    current = [current[ix].combine(current[ix + 1]) if ix + 1 < len(current) else StackMonoid(0, []).combine(current[ix]) for ix in range(0, len(current), 2) ]
    states.append(copy(current))
    
print_out_states([], states)

NameError: name 'StackMonoid' is not defined